## **Example run with mlflow.prophet**

This examples uses mlflow built in *mlflow.prophet.log_model* to log the model

Example is copied from mlflow offical github https://github.com/mlflow/mlflow/blob/master/examples/prophet/train.py

In [ ]:
conda activate mlflow_testing

In [1]:
import json

import numpy as np
import pandas as pd
from prophet import Prophet, serialize
from prophet.diagnostics import cross_validation, performance_metrics

import mlflow
from mlflow.models import infer_signature

SOURCE_DATA = (
    "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_retail_sales.csv"
)
ARTIFACT_PATH = "model"
np.random.seed(12345)

Importing plotly failed. Interactive plots will not work.


In [2]:
def extract_params(pr_model):
    return {attr: getattr(pr_model, attr) for attr in serialize.SIMPLE_ATTRIBUTES}

sales_data = pd.read_csv(SOURCE_DATA)

In [3]:
sales_data.head()

,ds,y
0,1992-01-01,146376
1,1992-02-01,147079
2,1992-03-01,159336
3,1992-04-01,163669
4,1992-05-01,170068


In [32]:
%env MLFLOW_TRACKING_URI=sqlite:///mlrun_prophet.db

env: MLFLOW_TRACKING_URI=sqlite:///mlrun_prophet.db


In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [ ]:
with mlflow.start_run():
    model = Prophet().fit(sales_data)

    params = extract_params(model)

    metric_keys = ["mse", "rmse", "mae", "mape", "mdape", "smape", "coverage"]
    metrics_raw = cross_validation(
        model=model,
        horizon="365 days",
        period="180 days",
        initial="710 days",
        parallel="threads",
        disable_tqdm=True,
    )
    cv_metrics = performance_metrics(metrics_raw)
    metrics = {k: cv_metrics[k].mean() for k in metric_keys}

    print(f"Logged Metrics: \n{json.dumps(metrics, indent=2)}")
    print(f"Logged Params: \n{json.dumps(params, indent=2)}")

    train = model.history
    predictions = model.predict(model.make_future_dataframe(30))
    signature = infer_signature(train, predictions)

    mlflow.prophet.log_model(model, artifact_path=ARTIFACT_PATH, signature=signature)
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    model_uri = mlflow.get_artifact_uri(ARTIFACT_PATH)
    print(f"Model artifact logged to: {model_uri}")


# loaded_model = mlflow.prophet.load_model(model_uri)

# forecast = loaded_model.predict(loaded_model.make_future_dataframe(60))

# print(f"forecast:\n${forecast.head(30)}")

After setting up the docker image and running the container (see more on notebook 4. Docker Quick Guide), we attempt to call the api

In short, you want to

**Connect to mlflow tracking uri**

`export MLFLOW_TRACKING_URI="http://127.0.0.1:8080"`

`$env:MLFLOW_TRACKING_URI="http://127.0.0.1:8080"`

**Build image**

`mlflow models build-docker --model-uri "models:/<model name>/<version>" --name "<image name>"`

**Run container**

`docker run -p 5002:8080 <image name>`

In [62]:
import requests

endpoint = "http://localhost:5002/invocations" # make sure the port number matches the one you input in terminal
data = {
    "dataframe_split": {  
        "columns": ['ds','y','floor','t','y_scaled'],
        "data": [['2016-06-01',0,0,0,0],['2016-06-01',10,10,10,120]],
        
    }
}
# do a post request
response = requests.post(endpoint, json=data)
print(response.json())

{'predictions': [{'ds': '2016-06-01T00:00:00', 'trend': 464314.1915900584, 'yhat_lower': 456857.610812237, 'yhat_upper': 477214.8479718002, 'trend_lower': 464314.1915900584, 'trend_upper': 464314.1915900584, 'additive_terms': 2862.888577091087, 'additive_terms_lower': 2862.888577091087, 'additive_terms_upper': 2862.888577091087, 'yearly': 2862.888577091087, 'yearly_lower': 2862.888577091087, 'yearly_upper': 2862.888577091087, 'multiplicative_terms': 0.0, 'multiplicative_terms_lower': 0.0, 'multiplicative_terms_upper': 0.0, 'yhat': 467177.0801671495}, {'ds': '2016-06-01T00:00:00', 'trend': 464314.1915900584, 'yhat_lower': 457352.15333530406, 'yhat_upper': 477562.4071601084, 'trend_lower': 464314.1915900584, 'trend_upper': 464314.1915900584, 'additive_terms': 2862.888577091087, 'additive_terms_lower': 2862.888577091087, 'additive_terms_upper': 2862.888577091087, 'yearly': 2862.888577091087, 'yearly_lower': 2862.888577091087, 'yearly_upper': 2862.888577091087, 'multiplicative_terms': 0.0,

**Noted that due to the nature of prophet, you will need to declare extra columns as demonstrated and pass in some random values inside (all zeros in this demonstration)**

## **Logging with pyfunc**

The idea here is to simply pass in a custom wrapper that add an extra pipeline `self.model.predict(model.make_future_dataframe(30))` so that user only need to input '30' to simplify the process

In [38]:
import json

import numpy as np
import pandas as pd
from prophet import Prophet, serialize
from prophet.diagnostics import cross_validation, performance_metrics

import mlflow
from mlflow.models import infer_signature

SOURCE_DATA = (
    "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_retail_sales.csv"
)
ARTIFACT_PATH = "model"
np.random.seed(12345)

sales_data = pd.read_csv(SOURCE_DATA)

In [5]:
%env MLFLOW_TRACKING_URI=sqlite:///prophet_mlrun.db

env: MLFLOW_TRACKING_URI=sqlite:///prophet_mlrun.db


In [6]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

**The key is to modify the predict() function**

In [107]:
import json

In [14]:
class ProphetWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = Prophet()
    def fit(self):
        self.model.fit(sales_data)
        train = self.model.history
        predictions = self.model.predict(model.make_future_dataframe(30))
        signature = infer_signature(pd.DataFrame({'ds': [len(train)]}), predictions)
        return train, predictions

    def predict(self, context, model_input):
        input_data = pd.DataFrame(model_input['ds']) # <-- the model input is what you pass in during API call, noted that this is fully customizable
        future = self.model.make_future_dataframe(periods=input_data.iloc[0,0])
        forecast = self.model.predict(future)
        return forecast.to_json() 

**Important: the result returned by predict() function should be in json format, instead of dataframe**

In [12]:
# you would also need to run a conda_env file if you have anything specific to install
import cloudpickle
conda_env = {
    'channels': ['defaults'],
    'dependencies': [
        'python=3.11.5',
        'pip',
          {
            'pip': [
                'mlflow',
                'pillow',
                'cloudpickle=={}'.format(cloudpickle.__version__),
                'cmdstanpy',
                'cmdstanpy',
                'cffi',
                'defusedxml',
                'holidays',
                'importlib-resources',
                'ipython',
                'matplotlib',
                'numpy',
                'packaging',
                'pandas',
                'prophet',
                'stanio',
                'tqdm'
                
            ],
          },
    ],
    'name': 'prophet_env'
}

**Below demonstrate the basic process of logging the custom wrappe**
1. call the wrapper + fit the model
2. log respective metadata (metrics/params/tags...)
3. log the artifact

In [ ]:
custom_model = ProphetWrapper()
# run the fit function to train the model
custom_model.fit()

with mlflow.start_run():
    # log the Python function model

    # Noted that you log the model metrics/features as well
    metric_keys = ["mse", "rmse", "mae", "mape", "mdape", "smape", "coverage"]
    metrics_raw = cross_validation(
        model=custom_model.model,
        horizon="365 days",
        period="180 days",
        initial="710 days",
        parallel="threads",
        disable_tqdm=True,
    )
    cv_metrics = performance_metrics(metrics_raw)
    metrics = {k: cv_metrics[k].mean() for k in metric_keys}
    
    # get parameters
    params = extract_params(custom_model.model)
    
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.pyfunc.log_model(
        python_model=custom_model,
        conda_env=conda_env,
        artifact_path="prophetwrapper",
        registered_model_name="prophetwrapper_model",
    )

**load the model for testing**

In [11]:
loaded_model1 = mlflow.pyfunc.load_model("models:/prophetwrapper_model/13")

In [127]:
# example of how you can use the model with model loading
a = pd.DataFrame({'ds': 30}, index=[0])
pd.DataFrame(json.loads(loaded_model1.predict(a)))

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,694224000000,162684.280977,116992.043108,137730.916329,162684.280977,162684.280977,-34235.732160,-34235.732160,-34235.732160,-34235.732160,-34235.732160,-34235.732160,0.0,0.0,0.0,128448.548817
1,696902400000,163738.429506,122261.343943,142745.683719,163738.429506,163738.429506,-30525.501333,-30525.501333,-30525.501333,-30525.501333,-30525.501333,-30525.501333,0.0,0.0,0.0,133212.928173
2,699408000000,164724.568452,158525.287039,179409.576037,164724.568452,164724.568452,4417.848692,4417.848692,4417.848692,4417.848692,4417.848692,4417.848692,0.0,0.0,0.0,169142.417144
3,702086400000,165778.716981,151955.287093,171539.935538,165778.716981,165778.716981,-3177.578486,-3177.578486,-3177.578486,-3177.578486,-3177.578486,-3177.578486,0.0,0.0,0.0,162601.138494
4,704678400000,166798.860718,169020.520277,188449.851232,166798.860718,166798.860718,12340.056279,12340.056279,12340.056279,12340.056279,12340.056279,12340.056279,0.0,0.0,0.0,179138.916997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,1464307200000,464118.007526,469364.866249,488587.406290,464118.007526,464118.007526,14683.607789,14683.607789,14683.607789,14683.607789,14683.607789,14683.607789,0.0,0.0,0.0,478801.615315
319,1464393600000,464164.339417,466598.171727,486185.294514,464164.339417,464164.339417,12102.932457,12102.932457,12102.932457,12102.932457,12102.932457,12102.932457,0.0,0.0,0.0,476267.271874
320,1464480000000,464210.671308,462824.072862,483889.298318,464210.671308,464210.671308,9568.611892,9568.611892,9568.611892,9568.611892,9568.611892,9568.611892,0.0,0.0,0.0,473779.283200
321,1464566400000,464257.003199,460892.155660,482491.828668,464257.003199,464257.003199,7142.929363,7142.929363,7142.929363,7142.929363,7142.929363,7142.929363,0.0,0.0,0.0,471399.932562


### Call the model through API

1. Build docker image
   
`mlflow models build-docker --model-uri "models:/prophetwrapper_model/13" --
name "prophetwrapper`

2. Run docker container

`docker run -p 5002:8080 -d prophetwrapper`

3. Make request

In [138]:
import requests

endpoint = "http://localhost:5002/invocations" # make sure the port number matches the one you input in terminal
data = {
    "dataframe_split": {
        "columns": ['ds'],
        "data": [[30]]
    }
}
# do a post request
response = requests.post(endpoint, json=data)
pd.DataFrame(json.loads(response.json()['predictions']))

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,694224000000,162684.280977,118794.133699,138014.399855,162684.280977,162684.280977,-34235.732160,-34235.732160,-34235.732160,-34235.732160,-34235.732160,-34235.732160,0.0,0.0,0.0,128448.548817
1,696902400000,163738.429506,122649.005189,143257.352002,163738.429506,163738.429506,-30525.501333,-30525.501333,-30525.501333,-30525.501333,-30525.501333,-30525.501333,0.0,0.0,0.0,133212.928173
2,699408000000,164724.568452,159787.694815,180223.583919,164724.568452,164724.568452,4417.848692,4417.848692,4417.848692,4417.848692,4417.848692,4417.848692,0.0,0.0,0.0,169142.417144
3,702086400000,165778.716981,152323.500569,172681.137653,165778.716981,165778.716981,-3177.578486,-3177.578486,-3177.578486,-3177.578486,-3177.578486,-3177.578486,0.0,0.0,0.0,162601.138494
4,704678400000,166798.860718,169694.426761,189828.984566,166798.860718,166798.860718,12340.056279,12340.056279,12340.056279,12340.056279,12340.056279,12340.056279,0.0,0.0,0.0,179138.916997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,1464307200000,464118.007526,468776.776924,488231.694856,464118.007526,464118.007526,14683.607789,14683.607789,14683.607789,14683.607789,14683.607789,14683.607789,0.0,0.0,0.0,478801.615315
319,1464393600000,464164.339417,465938.010143,487108.179089,464164.339417,464164.339417,12102.932457,12102.932457,12102.932457,12102.932457,12102.932457,12102.932457,0.0,0.0,0.0,476267.271874
320,1464480000000,464210.671308,462254.623159,484070.505996,464210.671308,464210.671308,9568.611892,9568.611892,9568.611892,9568.611892,9568.611892,9568.611892,0.0,0.0,0.0,473779.283200
321,1464566400000,464257.003199,461062.971012,481648.281918,464257.003199,464257.003199,7142.929363,7142.929363,7142.929363,7142.929363,7142.929363,7142.929363,0.0,0.0,0.0,471399.932562
